In [91]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import numpy as np
import string
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import random
from sklearn.utils import shuffle
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load
from sklearn.preprocessing import normalize
import gensim

def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    temp = text.replace("\\r", " ")
    temp = temp.replace("\\n", " ")
    return [stemmer.stem(w) for w in word_tokenize(temp)]

def save_model(model, filename='default.joblib'):
    dump(model, filename)
    
def load_model(filename='default.joblib'):
    return load(filename)

random.seed(42)

stop_words = stopwords.words('english')+list(string.punctuation)
stemmed_stop_words = stemming_tokenizer(" ".join(stop_words))

In [ ]:
load_binary = True
if load_binary:
    model = load_model('word2vec.joblib')
else:
    # https://github.com/alexandres/lexvec - https://www.dropbox.com/s/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz?dl=1
    model = gensim.models.KeyedVectors.load_word2vec_format('lexvec.commoncrawl.300d.W.pos.neg3.vectors', binary=False)
    save_model(model, 'word2vec.joblib')

In [93]:
def get_vector(text, debug=False):
    vec = np.zeros(300)
    found_count = 0
    total_count = 0
    for word in stemming_tokenizer(text):
        if word.lower() in stemmed_stop_words:
            continue
        total_count += 1
        if word.lower() in model:
            vec = vec + model[word.lower()]
            found_count += 1
    if debug:
        print(found_count, total_count)
    return vec

In [94]:
# load data
data = pd.read_csv('..\miniature.csv', na_filter=False)

In [95]:
# Create vectors
X_issue_title_wordvec = [get_vector(text, True) for text in data['IssueTitle']]
X_issue_description_wordvec = [get_vector(text, True) for text in data['IssueDescription']]
X_issue_label_wordvec = [get_vector(text, True) for text in data['Label']]
Y_pr_title_wordvec = [get_vector(text, True) for text in data['PrTitle']]
Y_pr_description_wordvec = [get_vector(text, True) for text in data['PrDescription']]

5 5
5 5
3 3
6 6
8 8
4 5
3 4
7 7
5 9
1 2
4 5
5 7
8 8
0 2
10 10
3 4
5 6
3 4
1 3
4 5
5 5
5 5
40 70
88 131
92 129
42 53
15 16
63 78
7 12
40 47
102 137
546 763
188 295
33 36
12 14
56 71
34 44
72 102
49 73
227 378
112 171
26 29
22 41
41 144
3 5
5 7
4 4
0 2
1 4
0 2
0 1
0 1
0 1
0 2
0 2
0 3
2 5
0 1
4 6
0 0
0 1
0 1
0 1
0 2
0 4
1 3
6 7
5 6
7 7
3 3
7 7
4 6
6 6
6 6
6 6
2 3
1 2
3 5
10 10
1 2
12 12
2 3
3 6
5 7
3 3
2 4
4 4
3 4
44 54
8 13
9 10
116 144
95 115
17 21
19 20
9 10
13 18
14 22
2 3
147 178
125 139
5 7
25 31
2 5
7 12
20 22
4 6
22 25
2 3
29 44


In [96]:
# Train-test split for all the data
X_issue_title_wordvec_train, X_issue_title_wordvec_test, X_issue_description_wordvec_train, X_issue_description_wordvec_test, X_issue_label_wordvec_train, X_issue_label_wordvec_test, Y_pr_title_wordvec_train, Y_pr_title_wordvec_test, Y_pr_description_wordvec_train, Y_pr_description_wordvec_test = train_test_split(X_issue_title_wordvec, X_issue_description_wordvec, X_issue_label_wordvec, Y_pr_title_wordvec, Y_pr_description_wordvec, test_size=0.25, random_state=33, shuffle=False)

In [97]:
# Model
classifier = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=500)
# to do: normalize before adding
def input_feature_extraction(x_title, x_desc):
    x1 = np.array(x_title)
    x2 = np.array(x_desc)
    a = 10
    b = 1
    return a*normalize(x1 - np.mean(x1, axis=0)) + b*normalize(x2 - np.mean(x2, axis=0))

def output_feature_extraction(y_title, y_desc):
    y1 = y_title
    y2 = y_desc
    a = 10
    b = 1
    return a*normalize(y1 - np.mean(y1, axis=0)) + b*normalize(y2 - np.mean(y2, axis=0))

train_input_features = input_feature_extraction(X_issue_title_wordvec_train, X_issue_description_wordvec_train)
train_output_features = output_feature_extraction(Y_pr_title_wordvec_train, Y_pr_description_wordvec_train)

classifier.fit(train_input_features, train_output_features)
modelFileName = "testmodel.joblib"
save_model(classifier, modelFileName)

In [88]:
# Prediction
fromFile = True
if fromFile:
    classifier = load_model(modelFileName)

def predict_and_get_cosine_sim(x, y, clf):
    y_pred = clf.predict(x)
    return cosine_similarity(y_pred, y)

predict_and_get_cosine_sim(train_input_features, train_output_features, classifier)

array([[ 0.99914582, -0.14782278, -0.13617297,  0.4005462 , -0.01778644,
         0.01244592, -0.11453562, -0.24562755, -0.05866832, -0.01213067,
         0.24078621, -0.21285879, -0.19324832,  0.23797231, -0.05188976,
         0.19092483],
       [-0.14874822,  0.99964869,  0.11289576, -0.24298703, -0.2333146 ,
        -0.04395603,  0.01249696,  0.0138356 ,  0.06788701, -0.17361683,
        -0.19640723, -0.07743124, -0.08183413, -0.15289761, -0.15232002,
        -0.19238332],
       [-0.13382612,  0.11344685,  0.99955797, -0.39762134, -0.13302158,
        -0.09391942, -0.09325713,  0.04338228,  0.15868882, -0.13977485,
        -0.37118556, -0.14764697, -0.04551899, -0.35315886,  0.034856  ,
        -0.3533675 ],
       [ 0.40616052, -0.2467188 , -0.40288243,  0.9982212 , -0.00443429,
        -0.07917863, -0.15181295, -0.22163781, -0.17907647,  0.25894716,
         0.46948477,  0.10786321, -0.18044567,  0.48657053, -0.18571154,
         0.4256713 ],
       [-0.01729129, -0.23529399, -0

In [98]:
def get_top_k_predictions(x, y, clf, k):
    cosineMat = predict_and_get_cosine_sim(x, y, clf)
    # Sort in descending order
    return np.argsort(-1*cosineMat)[:, :k]

# Test with PRs (train+test)
get_top_k_predictions(input_feature_extraction(X_issue_title_wordvec, X_issue_description_wordvec), output_feature_extraction(Y_pr_title_wordvec, Y_pr_description_wordvec), classifier, 4)

array([[ 0, 18,  3, 17],
       [ 1, 16,  2,  8],
       [ 2, 16,  8,  1],
       [ 3, 13, 10, 15],
       [ 4, 14, 12, 15],
       [ 5, 20, 12, 19],
       [ 6, 20, 19,  1],
       [ 7, 16, 11,  2],
       [ 8,  2, 17, 16],
       [ 9, 10, 13, 15],
       [10, 13, 15, 18],
       [11, 16,  7, 13],
       [12,  4,  5, 14],
       [13, 10, 15, 18],
       [14,  4,  2, 20],
       [15, 13, 10, 18],
       [ 1,  6,  2,  5],
       [13, 10, 15, 18],
       [13, 10, 15, 18],
       [13, 10, 15, 18],
       [ 9, 11,  3,  8],
       [ 9, 10, 13, 18]], dtype=int64)

In [90]:
# to do - metric for performance